# RGU-IIT CBR - Results evaluation

## Download Libraries

In [ ]:
!pip install datasets peft -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 82.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━

In [ ]:
!pip install -U angle-emb -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.3/195.3 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.7 MB/s eta 0:00:00


## Import Libraries

In [ ]:
from transformers import BertTokenizer, BertModel, AutoTokenizer, AutoModel
import torch

In [ ]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig

from angle_emb import AnglE, Prompts

In [ ]:
import torch

In [ ]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Load Dataset - Augmented Australia Legal QA

https://huggingface.co/datasets/Ramitha/open-australian-legal-qa-simplified-sent-tokenized

In [ ]:
df = pd.read_parquet("https://huggingface.co/datasets/Ramitha/open-australian-legal-qa-simplified-sent-tokenized/resolve/main/data/train-00000-of-00001.parquet?download=true")

In [ ]:
df_test = pd.read_parquet("https://huggingface.co/datasets/Ramitha/open-australian-legal-qa-test/resolve/main/data/test-00000-of-00001.parquet?download=true")

## Simple EDA

In [ ]:
df.head()

,question,answer,snippet,snippet_sentences,keywords,__index_level_0__,referenced_acts
0,In the case of Nasr v NRMA Insurance [2006] NS...,"In Nasr v NRMA Insurance [2006] NSWSC 1018, th...",3 The plaintiff claims that he was overseas w...,[ 3 The plaintiff claims that he was overseas ...,NRMA 6 July 2004 4 October 2005 8 June 2006 China,0,None
1,"In the case of R v NGUYEN [2001] NSWCCA 334, w...","In the case of R v NGUYEN [2001] NSWCCA 334, t...",29 The Appellant also gave evidence that she ...,[ 29 The Appellant also gave evidence that she...,Appellant Mr Nguyen cousin girlfriend heroin f...,1,None
2,In the case of Moore v Scenic Tours Pty Ltd [2...,In the case of Moore v Scenic Tours Pty Ltd [2...,Medium Neutral Citation: Moore v Scenic Tours ...,[Medium Neutral Citation: Moore v Scenic Tours...,Moore v Scenic Tours Pty Ltd [2015] NSWSC 237 ...,2,Civil Procedure Act 2005\nCorporations Regulat...
3,What were the circumstances and outcomes of th...,In the case of Inspector Phillip Estreich v Ha...,CITATION: Inspector Phillip Estreich v Hannas...,[ CITATION: Inspector Phillip Estreich v Hanna...,Inspector Phillip Estreich Hannas Civil Engine...,3,OCCUPATIONAL HEALTH AND SAFETY ACT 2000 - s 8(...
4,In the case of Ruddock v Vadarlis [2001] FCA 1...,"In Ruddock v Vadarlis [2001] FCA 1329, the Com...",206 It was submitted for the Commonwealth tha...,[ 206 It was submitted for the Commonwealth th...,Commonwealth VCCL Vadarlis North J habeas corp...,4,habeas corpus


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2084 entries, 0 to 2083
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   question           2084 non-null   object
 1   answer             2084 non-null   object
 2   snippet            2084 non-null   object
 3   snippet_sentences  2084 non-null   object
 4   keywords           2084 non-null   object
 5   __index_level_0__  2084 non-null   int64 
 6   referenced_acts    2084 non-null   object
dtypes: int64(1), object(6)
memory usage: 114.1+ KB


In [ ]:
question_list = list(set(df["question"]))
answer_list = list(set(df["answer"]))
main_snippet_list = list(set(df["snippet"]))
main_snippet_sentence_list = [ x.tolist() for x in df["snippet_sentences"].to_list()]

In [ ]:
len(answer_list), len(question_list), len(main_snippet_list)

(2084, 2084, 2082)

In [ ]:
question_answer_pairs = dict(zip(list(df["question"]), list(df["answer"])))

In [ ]:
snippet_question_pairs = dict(zip(list(df["snippet"]), list(df["question"])))

## Embedding Assets

In [ ]:
legal_tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
legal_model = AutoModel.from_pretrained("nlpaueb/legal-bert-base-uncased")

normal_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
normal_model = BertModel.from_pretrained("bert-base-uncased")

angle_model_id = 'SeanLee97/angle-bert-base-uncased-nli-en-v1'
angle_tokenizer = AutoTokenizer.from_pretrained(angle_model_id)
angle_model = AutoModel.from_pretrained(angle_model_id).cuda()

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/708 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
def get_legal_bert_embeddings(sentence, prompt):
    text = prompt.format(text=sentence)
    inputs = legal_tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    with torch.no_grad():
        outputs = legal_model(**inputs)

    last_hidden_states = outputs.last_hidden_state
    sentence_embedding = torch.mean(last_hidden_states, dim=1)

    return sentence_embedding[0].tolist()

In [ ]:
def get_normal_bert_embeddings(sentence, prompt):
    text = prompt.format(text=sentence)
    input_ids = normal_tokenizer.encode(text, add_special_tokens=True, truncation=True, max_length=512)
    input_ids = torch.tensor(input_ids).unsqueeze(0)  # Add batch dimension

    with torch.no_grad():
        outputs = normal_model(input_ids)
        embeddings = outputs.last_hidden_state.mean(dim=1).squeeze()  # Take mean of token embeddings

    return embeddings.tolist()

In [ ]:
def get_angle_bert_embeddings(sentence, prompt):
    text = prompt.format(text=sentence)
    input_ids = angle_tokenizer([text], return_tensors='pt', truncation=True, max_length=512)

    for k, v in input_ids.items():
      input_ids[k] = v.cuda()

    hidden_state = angle_model(**input_ids).last_hidden_state
    vec = (hidden_state[:, 0] + torch.mean(hidden_state, dim=1)) / 2.0

    return vec[0].tolist()

In [ ]:
# Test Embeddings:
legal_sentence = "The court dismisses the case due to lack of evidence."
normal_sentence = "This is a normal sentence for BERT embeddings."

empty_prompt = "{text}"

legal_embeddings = get_legal_bert_embeddings(normal_sentence, empty_prompt)
normal_embeddings = get_normal_bert_embeddings(normal_sentence, empty_prompt)
angle_embeddings = get_angle_bert_embeddings(normal_sentence, empty_prompt)

print("Legal BERT Embeddings:", len(legal_embeddings))
print("Normal BERT Embeddings:", len(normal_embeddings))
print("Angle BERT Embeddings:", len(angle_embeddings))

Legal BERT Embeddings: 768
Normal BERT Embeddings: 768
Angle BERT Embeddings: 768


In [ ]:
PROMPT_DICT = {
    "Matching": "{text}",
    "Retrieval_A": Prompts.A,
    "Retrieval_B": Prompts.B,
    "Retrieval_C": Prompts.C,
}

In [ ]:
PROMPT_DICT["Retrieval_C"]

'Represent this sentence for searching relevant passages: {text}'

## Get Embeddings

In [ ]:
!mkdir assets

In [ ]:
df.head()

,question,answer,snippet,snippet_sentences,keywords,__index_level_0__,referenced_acts
0,In the case of Nasr v NRMA Insurance [2006] NS...,"In Nasr v NRMA Insurance [2006] NSWSC 1018, th...",3 The plaintiff claims that he was overseas w...,[ 3 The plaintiff claims that he was overseas ...,NRMA 6 July 2004 4 October 2005 8 June 2006 China,0,None
1,"In the case of R v NGUYEN [2001] NSWCCA 334, w...","In the case of R v NGUYEN [2001] NSWCCA 334, t...",29 The Appellant also gave evidence that she ...,[ 29 The Appellant also gave evidence that she...,Appellant Mr Nguyen cousin girlfriend heroin f...,1,None
2,In the case of Moore v Scenic Tours Pty Ltd [2...,In the case of Moore v Scenic Tours Pty Ltd [2...,Medium Neutral Citation: Moore v Scenic Tours ...,[Medium Neutral Citation: Moore v Scenic Tours...,Moore v Scenic Tours Pty Ltd [2015] NSWSC 237 ...,2,Civil Procedure Act 2005\nCorporations Regulat...
3,What were the circumstances and outcomes of th...,In the case of Inspector Phillip Estreich v Ha...,CITATION: Inspector Phillip Estreich v Hannas...,[ CITATION: Inspector Phillip Estreich v Hanna...,Inspector Phillip Estreich Hannas Civil Engine...,3,OCCUPATIONAL HEALTH AND SAFETY ACT 2000 - s 8(...
4,In the case of Ruddock v Vadarlis [2001] FCA 1...,"In Ruddock v Vadarlis [2001] FCA 1329, the Com...",206 It was submitted for the Commonwealth tha...,[ 206 It was submitted for the Commonwealth th...,Commonwealth VCCL Vadarlis North J habeas corp...,4,habeas corpus


In [ ]:
df.columns

Index(['question', 'answer', 'snippet', 'snippet_sentences', 'keywords',
       '__index_level_0__', 'referenced_acts'],
      dtype='object')

In [ ]:
df.iloc[0]['snippet_sentences'][0]

' 3 The plaintiff claims that he was overseas when the Local Court struck out his case against the NRMA and they (the NRMA) rejected payment of his claim for his car after it was burnt on 6 July 2004.'

In [ ]:
new_df = pd.DataFrame(columns=df.columns)

In [ ]:
df_rows = df.copy()

In [ ]:
snippet_sentence_normal_bert_matching_embeddings = {}
snippet_sentence_legal_bert_matching_embeddings = {}
snippet_sentence_angle_bert_matching_embeddings = {}

snippet_sentence_normal_bert_retrieval_embeddings = {}
snippet_sentence_legal_bert_retrieval_embeddings = {}
snippet_sentence_angle_bert_retrieval_embeddings = {}

In [ ]:
i = 0
for index, row in tqdm(list(df_rows.iterrows())):
  i+=1
  try:
    # Question get Matching and Retreival Embeddings
    question_normal_bert_matching_embeddings = get_normal_bert_embeddings(row['question'], PROMPT_DICT["Matching"])
    question_legal_bert_matching_embeddings = get_legal_bert_embeddings(row['question'], PROMPT_DICT["Matching"])
    question_angle_bert_matching_embeddings = get_angle_bert_embeddings(row['question'], PROMPT_DICT["Matching"])

    question_normal_bert_retrieval_embeddings = get_normal_bert_embeddings(row['question'], PROMPT_DICT["Retrieval_C"])
    question_legal_bert_retrieval_embeddings = get_legal_bert_embeddings(row['question'], PROMPT_DICT["Retrieval_C"])
    question_angle_bert_retrieval_embeddings = get_angle_bert_embeddings(row['question'], PROMPT_DICT["Retrieval_C"])

    # Answer get Matching and Retreival Embeddings
    answer_normal_bert_matching_embeddings = get_normal_bert_embeddings(row['answer'], PROMPT_DICT["Matching"])
    answer_legal_bert_matching_embeddings = get_legal_bert_embeddings(row['answer'], PROMPT_DICT["Matching"])
    answer_angle_bert_matching_embeddings = get_angle_bert_embeddings(row['answer'], PROMPT_DICT["Matching"])

    answer_normal_bert_retrieval_embeddings = get_normal_bert_embeddings(row['answer'], PROMPT_DICT["Retrieval_C"])
    answer_legal_bert_retrieval_embeddings = get_legal_bert_embeddings(row['answer'], PROMPT_DICT["Retrieval_C"])
    answer_angle_bert_retrieval_embeddings = get_angle_bert_embeddings(row['answer'], PROMPT_DICT["Retrieval_C"])

    # Keywords get Matching and Retreival Embeddings
    keywords_normal_bert_matching_embeddings = get_normal_bert_embeddings(row['keywords'], PROMPT_DICT["Matching"])
    keywords_legal_bert_matching_embeddings = get_legal_bert_embeddings(row['keywords'], PROMPT_DICT["Matching"])
    keywords_angle_bert_matching_embeddings = get_angle_bert_embeddings(row['keywords'], PROMPT_DICT["Matching"])

    keywords_normal_bert_retrieval_embeddings = get_normal_bert_embeddings(row['keywords'], PROMPT_DICT["Retrieval_C"])
    keywords_legal_bert_retrieval_embeddings = get_legal_bert_embeddings(row['keywords'], PROMPT_DICT["Retrieval_C"])
    keywords_angle_bert_retrieval_embeddings = get_angle_bert_embeddings(row['keywords'], PROMPT_DICT["Retrieval_C"])

    # Snippet get Matching and Retreival Embeddings
    snippet_normal_bert_matching_embeddings = get_normal_bert_embeddings(row['snippet'], PROMPT_DICT["Matching"])
    snippet_legal_bert_matching_embeddings = get_legal_bert_embeddings(row['snippet'], PROMPT_DICT["Matching"])
    snippet_angle_bert_matching_embeddings = get_angle_bert_embeddings(row['snippet'], PROMPT_DICT["Matching"])

    snippet_normal_bert_retrieval_embeddings = get_normal_bert_embeddings(row['snippet'], PROMPT_DICT["Retrieval_C"])
    snippet_legal_bert_retrieval_embeddings = get_legal_bert_embeddings(row['snippet'], PROMPT_DICT["Retrieval_C"])
    snippet_angle_bert_retrieval_embeddings = get_angle_bert_embeddings(row['snippet'], PROMPT_DICT["Retrieval_C"])

    # Add Question Matching and Retreival Embeddings to row
    row["question_normal_bert_matching_embeddings"] = question_normal_bert_matching_embeddings
    row["question_legal_bert_matching_embeddings"] = question_legal_bert_matching_embeddings
    row["question_angle_bert_matching_embeddings"] = question_angle_bert_matching_embeddings

    row["question_normal_bert_retrieval_embeddings"] = question_normal_bert_retrieval_embeddings
    row["question_legal_bert_retrieval_embeddings"] = question_legal_bert_retrieval_embeddings
    row["question_angle_bert_retrieval_embeddings"] = question_angle_bert_retrieval_embeddings

    # Add Answer Matching and Retreival Embeddings to row
    row["answer_normal_bert_matching_embeddings"] = answer_normal_bert_matching_embeddings
    row["answer_legal_bert_matching_embeddings"] = answer_legal_bert_matching_embeddings
    row["answer_angle_bert_matching_embeddings"] = answer_angle_bert_matching_embeddings

    row["answer_normal_bert_retrieval_embeddings"] = answer_normal_bert_retrieval_embeddings
    row["answer_legal_bert_retrieval_embeddings"] = answer_legal_bert_retrieval_embeddings
    row["answer_angle_bert_retrieval_embeddings"] = answer_angle_bert_retrieval_embeddings

    # Add Keywords Matching and Retreival Embeddings to row
    row["keyword_normal_bert_matching_embeddings"] = keywords_normal_bert_matching_embeddings
    row["keyword_legal_bert_matching_embeddings"] = keywords_legal_bert_matching_embeddings
    row["keyword_angle_bert_matching_embeddings"] = keywords_angle_bert_matching_embeddings

    row["keyword_normal_bert_retrieval_embeddings"] = keywords_normal_bert_retrieval_embeddings
    row["keyword_legal_bert_retrieval_embeddings"] = keywords_legal_bert_retrieval_embeddings
    row["keyword_angle_bert_retrieval_embeddings"] = keywords_angle_bert_retrieval_embeddings

    # Add Snippet Matching and Retreival Embeddings to row
    row["snippet_normal_bert_matching_embeddings"] = snippet_normal_bert_matching_embeddings
    row["snippet_legal_bert_matching_embeddings"] = snippet_legal_bert_matching_embeddings
    row["snippet_angle_bert_matching_embeddings"] = snippet_angle_bert_matching_embeddings

    row["snippet_normal_bert_retrieval_embeddings"] = snippet_normal_bert_retrieval_embeddings
    row["snippet_legal_bert_retrieval_embeddings"] = snippet_legal_bert_retrieval_embeddings
    row["snippet_angle_bert_retrieval_embeddings"] = snippet_angle_bert_retrieval_embeddings

    for snippet_sentence in row['snippet_sentences']:
      # Add Snippet Sentence Matching and Retreival Embeddings to correspoding diconaries
      snippet_sentence_normal_bert_matching_embeddings[snippet_sentence] = get_normal_bert_embeddings(snippet_sentence, PROMPT_DICT["Matching"])
      snippet_sentence_legal_bert_matching_embeddings[snippet_sentence] = get_legal_bert_embeddings(snippet_sentence, PROMPT_DICT["Matching"])
      snippet_sentence_angle_bert_matching_embeddings[snippet_sentence] = get_angle_bert_embeddings(snippet_sentence, PROMPT_DICT["Matching"])

      snippet_sentence_normal_bert_retrieval_embeddings[snippet_sentence] = get_normal_bert_embeddings(snippet_sentence, PROMPT_DICT["Retrieval_C"])
      snippet_sentence_legal_bert_retrieval_embeddings[snippet_sentence] = get_legal_bert_embeddings(snippet_sentence, PROMPT_DICT["Retrieval_C"])
      snippet_sentence_angle_bert_retrieval_embeddings[snippet_sentence] = get_angle_bert_embeddings(snippet_sentence, PROMPT_DICT["Retrieval_C"])

    new_df = new_df.append(row, ignore_index=True)

  except Exception as e:
    print(e)
    print(f"Error in {i}")

100%|██████████| 2084/2084 [2:26:59<00:00,  4.23s/it]


In [ ]:
new_df.head()

,question,answer,snippet,snippet_sentences,keywords,__index_level_0__,referenced_acts,question_normal_bert_matching_embeddings,question_legal_bert_matching_embeddings,question_angle_bert_matching_embeddings,...,keyword_angle_bert_matching_embeddings,keyword_normal_bert_retrieval_embeddings,keyword_legal_bert_retrieval_embeddings,keyword_angle_bert_retrieval_embeddings,snippet_normal_bert_matching_embeddings,snippet_legal_bert_matching_embeddings,snippet_angle_bert_matching_embeddings,snippet_normal_bert_retrieval_embeddings,snippet_legal_bert_retrieval_embeddings,snippet_angle_bert_retrieval_embeddings
0,In the case of Nasr v NRMA Insurance [2006] NS...,"In Nasr v NRMA Insurance [2006] NSWSC 1018, th...",3 The plaintiff claims that he was overseas w...,[ 3 The plaintiff claims that he was overseas ...,NRMA 6 July 2004 4 October 2005 8 June 2006 China,0,None,"[0.0952058956027031, -0.41586318612098694, -0....","[-0.15682467818260193, 0.09918645769357681, -0...","[-0.44755819439888, -0.30833685398101807, -0.0...",...,"[-0.7435569167137146, 0.36318933963775635, 0.2...","[-0.3109404742717743, 0.07228869944810867, 0.1...","[-0.10291731357574463, 0.29612767696380615, 0....","[-0.7065484523773193, 0.4063403606414795, 0.11...","[-0.3986404240131378, -0.06476008147001266, 0....","[-0.12003172188997269, 0.31072312593460083, 0....","[-0.38614457845687866, 0.2507633566856384, 0.0...","[-0.378875195980072, -0.05569363757967949, 0.1...","[-0.10320620238780975, 0.31276533007621765, -0...","[-0.387744665145874, 0.26941773295402527, 0.01..."
1,"In the case of R v NGUYEN [2001] NSWCCA 334, w...","In the case of R v NGUYEN [2001] NSWCCA 334, t...",29 The Appellant also gave evidence that she ...,[ 29 The Appellant also gave evidence that she...,Appellant Mr Nguyen cousin girlfriend heroin f...,1,None,"[0.10903532803058624, -0.07590213418006897, -0...","[-0.09973642975091934, 0.11940381675958633, -0...","[-0.44240397214889526, 0.22697538137435913, -0...",...,"[-0.45839327573776245, 0.4812496304512024, 0.3...","[0.2508794367313385, 0.12569917738437653, 0.31...","[-0.020035812631249428, 0.36534440517425537, 0...","[-0.45669496059417725, 0.5146573185920715, 0.2...","[-0.17486847937107086, -0.06507079303264618, 0...","[-0.15001508593559265, -0.02271934598684311, 0...","[-0.2910490334033966, -0.13354447484016418, -0...","[-0.16949313879013062, -0.04134415462613106, 0...","[-0.14702622592449188, -0.012033158913254738, ...","[-0.2847302556037903, -0.07715550065040588, -0..."
2,In the case of Moore v Scenic Tours Pty Ltd [2...,In the case of Moore v Scenic Tours Pty Ltd [2...,Medium Neutral Citation: Moore v Scenic Tours ...,[Medium Neutral Citation: Moore v Scenic Tours...,Moore v Scenic Tours Pty Ltd [2015] NSWSC 237 ...,2,Civil Procedure Act 2005\nCorporations Regulat...,"[0.28107333183288574, -0.27589425444602966, -0...","[0.02962103858590126, 0.010986064560711384, -0...","[0.12405728548765182, -0.2887035012245178, 0.4...",...,"[-0.36992913484573364, -0.1369560956954956, 0....","[-0.2607484757900238, 0.03614029288291931, 0.3...","[-0.20989347994327545, 0.19033683836460114, 0....","[-0.379056453704834, -0.08602993935346603, 0.2...","[-0.2752224802970886, 0.014924434944987297, 0....","[-0.21225814521312714, 0.17278948426246643, 0....","[-0.4051317572593689, -0.1348607987165451, 0.3...","[-0.2772049903869629, 0.038114968687295914, 0....","[-0.18782252073287964, 0.17691953480243683, 0....","[-0.43150240182876587, -0.048282869160175323, ..."
3,What were the circumstances and outcomes of th...,In the case of Inspector Phillip Estreich v Ha...,CITATION: Inspector Phillip Estreich v Hannas...,[ CITATION: Inspector Phillip Estreich v Hanna...,Inspector Phillip Estreich Hannas Civil Engine...,3,OCCUPATIONAL HEALTH AND SAFETY ACT 2000 - s 8(...,"[0.06449601799249649, -0.11428474634885788, -0...","[0.1003992035984993, 0.10484790056943893, -0.0...","[-0.2814714312553406, -0.15697334706783295, -0...",...,"[0.08131945133209229, -0.08465023338794708, 0....","[0.00693

In [ ]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2084 entries, 0 to 2083
Data columns (total 31 columns):
 #   Column                                     Non-Null Count  Dtype 
---  ------                                     --------------  ----- 
 0   question                                   2084 non-null   object
 1   answer                                     2084 non-null   object
 2   snippet                                    2084 non-null   object
 3   snippet_sentences                          2084 non-null   object
 4   keywords                                   2084 non-null   object
 5   __index_level_0__                          2084 non-null   object
 6   referenced_acts                            2084 non-null   object
 7   question_normal_bert_matching_embeddings   2084 non-null   object
 8   question_legal_bert_matching_embeddings    2084 non-null   object
 9   question_angle_bert_matching_embeddings    2084 non-null   object
 10  question_normal_bert_retrieval_embed

In [ ]:
def save_dict_as_json(dictionary, file_path):
  with open(file_path, 'w') as json_file:
    json.dump(dictionary, json_file)

In [ ]:
# Save all snippet sentnce diconaries to assets
save_dict_as_json(snippet_sentence_normal_bert_matching_embeddings, './assets/snippet_sentences_normal_bert_matching_embeddings.json')
save_dict_as_json(snippet_sentence_legal_bert_matching_embeddings, './assets/snippet_sentences_legal_bert_matching_embeddings.json')
save_dict_as_json(snippet_sentence_angle_bert_matching_embeddings, './assets/snippet_sentences_angle_bert_matching_embeddings.json')

save_dict_as_json(snippet_sentence_normal_bert_retrieval_embeddings, './assets/snippet_sentences_normal_bert_retrieval_embeddings.json')
save_dict_as_json(snippet_sentence_legal_bert_retrieval_embeddings, './assets/snippet_sentences_legal_bert_retrieval_embeddings.json')
save_dict_as_json(snippet_sentence_angle_bert_retrieval_embeddings, './assets/snippet_sentences_angle_bert_retrieval_embeddings.json')

In [ ]:
new_df.to_csv("assets/main.csv", index=False)

### Augmented Questions

In [ ]:
aug_df = pd.read_csv("https://huggingface.co/datasets/imperialwarrior/australia-legal-qa-dataset-and-embeddings/resolve/main/main.csv?download=true")

In [ ]:
aug_df.head()

,question,answer,text,prompt,source,question_non_retrieval_embeddings,answer_retrieval_embeddings,answer_non_retrieval_embeddings,question_retrieval_embeddings,text_non_retrieval_embeddings,text_retrieval_embeddings
0,In the case of Nasr v NRMA Insurance [2006] NS...,"In Nasr v NRMA Insurance [2006] NSWSC 1018, th...",Question: In the case of Nasr v NRMA Insurance...,# Snippet\nThe snippet from an Australian lega...,{'citation': 'Nasr v NRMA Insurance [2006] NSW...,"[0.0927734375, -0.73046875, 0.2080078125, 0.21...","[1.5, 0.8203125, 2.703125, -1.0546875, -1.6718...","[-1.0234375, -0.48828125, 0.0927734375, 1.5468...","[0.79296875, 0.185546875, 1.1015625, -0.392578...","[-0.6328125, -0.3046875, 0.328125, 1.28125, -1...","[1.1640625, 0.359375, 2.359375, -0.73046875, -..."
1,What was the reason for the plaintiff's appeal...,"In Nasr v NRMA Insurance [2006] NSWSC 1018, th...",Question: In the case of Nasr v NRMA Insurance...,# Snippet\nThe snippet from an Australian lega...,{'citation': 'Nasr v NRMA Insurance [2006] NSW...,"[-0.08740234375, -1.5703125, 0.7578125, 0.6914...","[1.5, 0.8203125, 2.703125, -1.0546875, -1.6718...","[-1.0234375, -0.48828125, 0.0927734375, 1.5468...","[0.609375, -0.2421875, 1.109375, -0.0187988281...","[-0.6328125, -0.3046875, 0.328125, 1.28125, -1...","[1.1640625, 0.359375, 2.359375, -0.73046875, -..."
2,Why was the plaintiff's appeal filed late in t...,"In Nasr v NRMA Insurance [2006] NSWSC 1018, th...",Question: In the case of Nasr v NRMA Insurance...,# Snippet\nThe snippet from an Australian lega...,{'citation': 'Nasr v NRMA Insurance [2006] NSW...,"[0.1279296875, -0.859375, 0.486328125, 0.07666...","[1.5, 0.8203125, 2.703125, -1.0546875, -1.6718...","[-1.0234375, -0.48828125, 0.0927734375, 1.5468...","[1.15625, -0.0098876953125, 1.1796875, -0.4257...","[-0.6328125, -0.3046875, 0.328125, 1.28125, -1...","[1.1640625, 0.359375, 2.359375, -0.73046875, -..."
3,Why was the plaintiff's appeal in the case Nas...,"In Nasr v NRMA Insurance [2006] NSWSC 1018, th...",Question: In the case of Nasr v NRMA Insurance...,# Snippet\nThe snippet from an Australian lega...,{'citation': 'Nasr v NRMA Insurance [2006] NSW...,"[-0.6328125, -2.03125, 0.56640625, 0.474609375...","[1.5, 0.8203125, 2.703125, -1.0546875, -1.6718...","[-1.0234375, -0.48828125, 0.0927734375, 1.5468...","[0.9296875, -0.20703125, 0.8828125, -0.3300781...","[-0.6328125, -0.3046875, 0.328125, 1.28125, -1...","[1.1640625, 0.359375, 2.359375, -0.73046875, -..."
4,Why was the plaintiff's appeal in the Nasr v N...,"In Nasr v NRMA Insurance [2006] NSWSC 1018, th...",Question: In the case of Nasr v NRMA Insurance...,# Snippet\nThe snippet from an Australian lega...,{'citation': 'Nasr v NRMA Insurance [2006] NSW...,"[0.01129150390625, -1.5390625, 0.6875, 0.57031...","[1.5, 0.8203125, 2.703125, -1.0546875, -1.6718...","[-1.0234375, -0.48828125, 0.0927734375, 1.5468...","[0.55859375, 0.039306640625, 1.3046875, -0.380...","[-0.6328125, -0.3046875, 0.328125, 1.28125, -1...","[1.1640625, 0.359375, 2.359375, -0.73046875, -..."


In [ ]:
len(aug_df["question"])

10620

In [ ]:
augmented_question_normal_bert_matching_embeddings = {}
augmented_question_legal_bert_matching_embeddings = {}
augmented_question_angle_bert_matching_embeddings = {}

augmented_question_normal_bert_retrieval_embeddings = {}
augmented_question_legal_bert_retrieval_embeddings = {}
augmented_question_angle_bert_retrieval_embeddings = {}

In [ ]:
i = 0
for index, row in tqdm(list(df_test.copy().iterrows())):
  i+=1
  try:
    # Question get Matching and Retreival Embeddings
    question_normal_bert_matching_embeddings = get_normal_bert_embeddings(row['question'], PROMPT_DICT["Matching"])
    question_legal_bert_matching_embeddings = get_legal_bert_embeddings(row['question'], PROMPT_DICT["Matching"])
    question_angle_bert_matching_embeddings = get_angle_bert_embeddings(row['question'], PROMPT_DICT["Matching"])

    question_normal_bert_retrieval_embeddings = get_normal_bert_embeddings(row['question'], PROMPT_DICT["Retrieval_C"])
    question_legal_bert_retrieval_embeddings = get_legal_bert_embeddings(row['question'], PROMPT_DICT["Retrieval_C"])
    question_angle_bert_retrieval_embeddings = get_angle_bert_embeddings(row['question'], PROMPT_DICT["Retrieval_C"])

    # Add Question Matching and Retreival Embeddings to dict
    augmented_question_normal_bert_matching_embeddings[row['question']] = question_normal_bert_matching_embeddings
    augmented_question_legal_bert_matching_embeddings[row['question']] = question_legal_bert_matching_embeddings
    augmented_question_angle_bert_matching_embeddings[row['question']] = question_angle_bert_matching_embeddings

    augmented_question_normal_bert_retrieval_embeddings[row['question']] = question_normal_bert_retrieval_embeddings
    augmented_question_legal_bert_retrieval_embeddings[row['question']] = question_legal_bert_retrieval_embeddings
    augmented_question_angle_bert_retrieval_embeddings[row['question']] = question_angle_bert_retrieval_embeddings

  except Exception as e:
    print(e)
    print(f"Error in {i}")

100%|██████████| 10620/10620 [1:17:59<00:00,  2.27it/s]


In [ ]:
# Save all snippet sentnce diconaries to files folder
save_dict_as_json(augmented_question_normal_bert_matching_embeddings, './files/augmented_questions_normal_bert_matching_embeddings.json')
save_dict_as_json(augmented_question_legal_bert_matching_embeddings, './files/augmented_questions_legal_bert_matching_embeddings.json')
save_dict_as_json(augmented_question_angle_bert_matching_embeddings, './files/augmented_questions_angle_bert_matching_embeddings.json')

save_dict_as_json(augmented_question_normal_bert_retrieval_embeddings, './files/augmented_questions_normal_bert_retrieval_embeddings.json')
save_dict_as_json(augmented_question_legal_bert_retrieval_embeddings, './files/augmented_questions_legal_bert_retrieval_embeddings.json')
save_dict_as_json(augmented_question_angle_bert_retrieval_embeddings, './files/augmented_questions_angle_bert_retrieval_embeddings.json')

## Test Dataset

In [ ]:
test_processed_df = pd.DataFrame(columns=df_test.columns)

In [ ]:
test_processed_df.head()

,question,answer,original_texts


In [ ]:
i = 0
for index, row in tqdm(list(df_test.copy().iterrows())):
  i+=1
  try:
    # Question get Matching and Retreival Embeddings
    question_normal_bert_matching_embeddings = get_normal_bert_embeddings(row['question'], PROMPT_DICT["Matching"])
    question_legal_bert_matching_embeddings = get_legal_bert_embeddings(row['question'], PROMPT_DICT["Matching"])
    question_angle_bert_matching_embeddings = get_angle_bert_embeddings(row['question'], PROMPT_DICT["Matching"])

    question_normal_bert_retrieval_embeddings = get_normal_bert_embeddings(row['question'], PROMPT_DICT["Retrieval_C"])
    question_legal_bert_retrieval_embeddings = get_legal_bert_embeddings(row['question'], PROMPT_DICT["Retrieval_C"])
    question_angle_bert_retrieval_embeddings = get_angle_bert_embeddings(row['question'], PROMPT_DICT["Retrieval_C"])

    # Answer get Matching and Retreival Embeddings
    answer_normal_bert_matching_embeddings = get_normal_bert_embeddings(row['answer'], PROMPT_DICT["Matching"])
    answer_legal_bert_matching_embeddings = get_legal_bert_embeddings(row['answer'], PROMPT_DICT["Matching"])
    answer_angle_bert_matching_embeddings = get_angle_bert_embeddings(row['answer'], PROMPT_DICT["Matching"])

    answer_normal_bert_retrieval_embeddings = get_normal_bert_embeddings(row['answer'], PROMPT_DICT["Retrieval_C"])
    answer_legal_bert_retrieval_embeddings = get_legal_bert_embeddings(row['answer'], PROMPT_DICT["Retrieval_C"])
    answer_angle_bert_retrieval_embeddings = get_angle_bert_embeddings(row['answer'], PROMPT_DICT["Retrieval_C"])

    # Add Question Matching and Retreival Embeddings to row
    row["question_normal_bert_matching_embeddings"] = question_normal_bert_matching_embeddings
    row["question_legal_bert_matching_embeddings"] = question_legal_bert_matching_embeddings
    row["question_angle_bert_matching_embeddings"] = question_angle_bert_matching_embeddings

    row["question_normal_bert_retrieval_embeddings"] = question_normal_bert_retrieval_embeddings
    row["question_legal_bert_retrieval_embeddings"] = question_legal_bert_retrieval_embeddings
    row["question_angle_bert_retrieval_embeddings"] = question_angle_bert_retrieval_embeddings

    # Add Answer Matching and Retreival Embeddings to row
    row["answer_normal_bert_matching_embeddings"] = answer_normal_bert_matching_embeddings
    row["answer_legal_bert_matching_embeddings"] = answer_legal_bert_matching_embeddings
    row["answer_angle_bert_matching_embeddings"] = answer_angle_bert_matching_embeddings

    row["answer_normal_bert_retrieval_embeddings"] = answer_normal_bert_retrieval_embeddings
    row["answer_legal_bert_retrieval_embeddings"] = answer_legal_bert_retrieval_embeddings
    row["answer_angle_bert_retrieval_embeddings"] = answer_angle_bert_retrieval_embeddings

    test_processed_df = test_processed_df.append(row, ignore_index=True)
  except Exception as e:
    print(e)
    print(f"Error in {i}")

100%|██████████| 35/35 [00:59<00:00,  1.69s/it]


In [ ]:
test_processed_df.to_csv("./files/main_test.csv", index=False)

In [ ]:
test_processed_df.to_csv("./assets/main_test.csv", index=False)

In [ ]:
test_processed_df

,question,answer,original_texts,question_normal_bert_matching_embeddings,question_legal_bert_matching_embeddings,question_angle_bert_matching_embeddings,question_normal_bert_retrieval_embeddings,question_legal_bert_retrieval_embeddings,question_angle_bert_retrieval_embeddings,answer_normal_bert_matching_embeddings,answer_legal_bert_matching_embeddings,answer_angle_bert_matching_embeddings,answer_normal_bert_retrieval_embeddings,answer_legal_bert_retrieval_embeddings,answer_angle_bert_retrieval_embeddings
0,In the case Econ Holdings Pty Ltd v Sims Lockw...,In the case Econ Holdings Pty Ltd v Sims Lockw...,"[114, 169]","[0.10423500835895538, -0.16651767492294312, -0...","[0.0027038531843572855, -0.12522685527801514, ...","[-0.3710246682167053, -0.30695846676826477, -0...","[0.09509201347827911, -0.20218567550182343, -0...","[-0.002354818396270275, -0.10421641916036606, ...","[-0.4049617350101471, -0.05314958840608597, -0...","[-0.2280706912279129, -0.09595099836587906, 0....","[-0.11683200299739838, -0.0331629179418087, 0....","[-0.4248591661453247, -0.04108070954680443, -0...","[-0.22192727029323578, -0.09357575327157974, 0...","[-0.09864509105682373, -0.024802161380648613, ...","[-0.4455259144306183, 0.04232563078403473, -0...."
1,Considering the balance of public interest and...,The details in Events 38832023 and 3860136 inc...,"[270, 347]","[-0.019197480753064156, -0.0016129742143675685...","[-0.12920249998569489, 0.08838286995887756, 0....","[0.624129056930542, -0.3914183974266052, -0.04...","[0.0265157800167799, -0.013784907758235931, -0...","[-0.05422680452466011, 0.12092851847410202, -0...","[0.36398348212242126, -0.16158992052078247, -0...","[-0.24886417388916016, 0.24751858413219452, -0...","[0.04563891515135765, 0.19787012040615082, -0....","[0.5655755996704102, -0.0976831465959549, -0.1...","[-0.2162676751613617, 0.23963819444179535, -0....","[0.05516554042696953, 0.20160460472106934, -0....","[0.5197416543960571, -0.056952767074108124, -0..."
2,In two separate cases heard in the Supreme Cou...,In the case of Mekhail v Hana; Mekail v Hana; ...,"[388, 1662]","[0.02596251852810383, -0.0713656097650528, -0....","[-0.03259794041514397, 0.28346192836761475, 0....","[-0.322487473487854, -0.10779520124197006, -0....","[0.06910748779773712, -0.13112029433250427, -0...","[-0.0018808066379278898, 0.26328015327453613, ...","[-0.3880188763141632, 0.0675685852766037, -0.7...","[-0.2993234694004059, -0.08719383180141449, 0....","[-0.15291669964790344, 0.12348964810371399, 0....","[-0.3596920967102051, -0.22549670934677124, -0...","[-0.29801949858665466, -0.0923399105668068, 0....","[-0.14582784473896027, 0.14794543385505676, 0....","[-0.379778653383255, -0.15069353580474854, -0...."
3,"In the context of the Federal Court Rules, how...","According to the judgment of Gordon J, an amen...","[1881, 1895]","[0.05399204418063164, 0.037635646760463715, -0...","[-0.09401072561740875, 0.03195436671376228, 0....","[-0.08438273519277573, -0.07412683963775635, -...","[0.05692090466618538, -0.04286135733127594, -0...","[-0.098471999168396, 0.03212614730000496, -0.0...","[-0.20830093324184418, 0.040662266314029694, -...","[-0.28019508719444275, 0.016026904806494713, -...","[-0.21366316080093384, 0.0026467808056622744, ...","[0.09185919910669327, 0.03754888102412224, -0....","[-0.2666361927986145, 0.0039128796197474, -0.0...","[-0.22142712771892548, 0.01649310067296028, -0...","[0.034823209047317505, 0.11204800009727478, -0..."
4,What are the specific models of Hiller UH-12 S...,"For the Hiller UH-12 Series Helicopters, the A...","[475, 762]","[0.02782502956688404, 0.30563706159591675, 0.2...","[-0.19913506507873535, 0.27476564049720764, 0....","[0.2879544496536255, 0.6913779973983765, 0.732...","[-0.0052728974260389805, 0.23753692209720612, ...","[-0.2159079909324646, 0.219609797000885, 0.193...","[0.2321493923664093, 0.7232281565666199, 0.550...","[-0.29812127351760864, -0.021036425605416298, ...","[-0.3260062634944916, 0.015874618664383888, 0